In [ ]:
# Install necessary library
!pip install geemap

In [ ]:
# Import necessary libraries
import ee
import geemap
import datetime

In [ ]:
# Create an interactive map
Map = geemap.Map()

In [ ]:
# Load the GAUL dataset (Second-Level Administrative Units)
gaul = ee.FeatureCollection("FAO/GAUL/2015/level2")

# Filter Sabak Bernam administrative unit
sabak = gaul.filter(ee.Filter.eq("ADM2_NAME", "Sabak Bernam"))

# Draw the boundary of Sabak Bernam
sabak_boundary = ee.Image().byte().paint(
  featureCollection = sabak,
  width=3
)

# Specify the visualisation of the boundary of Sabak Bernam
sabak_vis = {
    "palette": "black"
}

# Add Sabak Bernam's boundary to the map as a layer
Map.addLayer(sabak_boundary, sabak_vis, "Sabak Bernam")

# Center the map on Sabak Bernam
Map.centerObject(sabak, 8)

# Display the map
Map

In [ ]:
# Selecting soil temperature in the Sabak Bernam area
soil_temp = (ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')
               .select('soil_temperature_level_1')
               .filterDate('2023-03-05', '2023-05-25')
               .filterBounds(sabak))

# Define the visualisation parameters
soil_temp_vis = {
  'min': 273,
  'max': 312,
  'palette': ['lightskyblue', 'mediumspringgreen', 'palegoldenrod', 'orange', 'red'],
}

image = soil_temp.median()

# Adding the image to the map as a layer
Map.addLayer(image.clip(sabak), soil_temp_vis, 'Soil Temperature')

#Display map
Map

In [ ]:
#bbox = m.user_roi_bounds()
# Sekinchan area
bbox=[101.0821, 3.5211, 101.1456, 3.5777]
print("Sekinchan Area:", bbox)

In [ ]:
# Define a function to calculate the mean soil temperature value for each image in the collection within the specified polygon
def calculate_mean_soil_temp(image):
    meanKelvin = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=ee.Geometry.BBox(bbox[0], bbox[1], bbox[2], bbox[3]),
        scale=11132,
        maxPixels=1e9
    ).get('soil_temperature_level_1')

    # Convert to Celsius from Kelvin
    meanCelsius = ee.Number(meanKelvin).subtract(273.15)

    # Create a feature with null geometry
    # Set soil temperature value and date as properties
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    feature = ee.Feature(None, {'date': date, 'soil_temp': meanCelsius})
    return feature

# Define start and end dates for the first quarter
start_date = '2019-03-05'
end_date = '2019-05-25'

# Initialise values
year = 1
quarter = 1

# Define the start and end dates for the second quarter
second_quarter_start_date = '2019-09-05'
second_quarter_end_date = '2019-11-25'

while (quarter <= 2):
    while (year <= 3):
        # Load the climate dataset
        collection = (ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')
          .select('soil_temperature_level_1')
          .filterDate(start_date, end_date)
          .filterBounds(sabak)
          .filterBounds(ee.Geometry.BBox(bbox[0], bbox[1], bbox[2], bbox[3])))

        # Map the function over the collection to calculate the mean soil temperature value for each image within the specified polygon
        time_series = collection.map(calculate_mean_soil_temp)

        # Define a name for the exported CSV file
        if (quarter==1):
          filename = "Soil Temperature_" + start_date[0:4] + "_Q1"
        elif (quarter==2):
          filename = "Soil Temperature_" + start_date[0:4] + "_Q2"

        # Export the features to a CSV file
        task = ee.batch.Export.table.toDrive(
            collection=time_series,
            description='soil_temperature',
            folder='soil_temp',
            fileNamePrefix= filename,
            fileFormat='CSV'
        )
        task.start()

        # Convert the start and end dates to datetime objects
        start_date_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
        end_date_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d')

        # Increment the year by 1
        start_date_dt = start_date_dt.replace(year=start_date_dt.year + 1)
        end_date_dt = end_date_dt.replace(year=end_date_dt.year + 1)

        # Convert the datetime objects back to strings
        start_date = start_date_dt.strftime('%Y-%m-%d')
        end_date = end_date_dt.strftime('%Y-%m-%d')

        year = year + 1

    quarter = quarter + 1

    # Set the start and end dates for the second quarter
    start_date = second_quarter_start_date
    end_date = second_quarter_end_date

    year = 1

In [ ]:
# Check task status
task.status()